In [1]:
import glob, os, cv2
import string
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import tensorflow.keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import time
# from draw_ellipse import *
# from ellipses import *
# from bwperim import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"]='2,3'
mirrored_strategy = tf.distribute.MirroredStrategy()



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [13]:
def build_BCDU_Net(filters=32, activation='relu', kernel_initializer='he_uniform', input_shape=(240,320,1)):
    FILTERS = filters
    ACTIVATION = activation
    KERNEL_INIT = kernel_initializer
    INPUT_SHAPE = input_shape
    H = input_shape[0]
    W = input_shape[1]
    
    inputs = Input(shape=INPUT_SHAPE)
    conv1 = Conv2D(filters=FILTERS, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation(ACTIVATION)(conv1)
    
    conv1 = Conv2D(filters=FILTERS, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation(ACTIVATION)(conv1)
    
    pool1 = MaxPooling2D()(conv1)
    
    conv2 = Conv2D(filters=FILTERS *2, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation(ACTIVATION)(conv2)
    
    conv2 = Conv2D(filters=FILTERS *2, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation(ACTIVATION)(conv2)
    
    pool2 = MaxPooling2D()(conv2)
    
    conv3 = Conv2D(filters=FILTERS *4, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation(ACTIVATION)(conv3)
    
    conv3 = Conv2D(filters=FILTERS *4, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation(ACTIVATION)(conv3)
    
    pool3 = MaxPooling2D()(conv3)

    conv4_1 = Conv2D(filters=FILTERS *8, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(pool3)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_1 = Activation(ACTIVATION)(conv4_1)
    
    conv4_2 = Conv2D(filters=FILTERS *8, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_2 = Activation(ACTIVATION)(conv4_2)
    
    conv4_3 = Conv2D(filters=FILTERS *8, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_3 = Activation(ACTIVATION)(conv4_3)
    
    conv4_4 = Conv2D(filters=FILTERS *8, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    conv4_4 = Activation(ACTIVATION)(conv4_4)
    
    merge_dense = concatenate([conv4_4, conv4_1], axis= -1)
    
    conv4_5 = Conv2D(filters=FILTERS *8, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(merge_dense)
    conv4_5 = BatchNormalization()(conv4_5)
    conv4_5 = Activation(ACTIVATION)(conv4_5)
    
    
    up5 = Conv2DTranspose(filters=FILTERS *4, kernel_size=2, strides=2, padding='same', kernel_initializer=KERNEL_INIT)(conv4_5)
    up5 = BatchNormalization()(up5)
    up5 = Activation(ACTIVATION)(up5)
    
    X1 = Reshape(target_shape=(1, np.int32(H/4), np.int32(W/4), FILTERS *4))(conv3)
    X2 = Reshape(target_shape=(1, np.int32(H/4), np.int32(W/4), FILTERS *4))(up5)
#     X1 = Reshape(target_shape=(1, np.int32(H/4), np.int32(W/4), 128))(conv3)
#     X2 = Reshape(target_shape=(1, np.int32(H/4), np.int32(W/4), 128))(up5)
    
    merge5 = concatenate([X1,X2], axis=1)
    merge5 = ConvLSTM2D(FILTERS *2, kernel_size=(3,3), padding='same', return_sequences=False, go_backwards=True, kernel_initializer=KERNEL_INIT)(merge5)
    
    conv5 = Conv2D(FILTERS *4, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(merge5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation(ACTIVATION)(conv5)
    
    conv5 = Conv2D(FILTERS *4, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation(ACTIVATION)(conv5)
    
    up6 = Conv2DTranspose(filters=FILTERS *2, kernel_size=2, strides=2, padding='same', kernel_initializer=KERNEL_INIT)(conv5)
    up6 = BatchNormalization(axis=3)(up6)
    up6 = Activation(ACTIVATION)(up6)
    
    X1 = Reshape(target_shape=(1, np.int32(H/2), np.int32(W/2), FILTERS *2))(conv2)
    X2 = Reshape(target_shape=(1, np.int32(H/2), np.int32(W/2), FILTERS *2))(up6)
#     X1 = Reshape(target_shape=(1, np.int32(H/2), np.int32(W/2), 64))(conv2)
#     X2 = Reshape(target_shape=(1, np.int32(H/2), np.int32(W/2), 64))(up6)
                               
    merge6 = concatenate([X1,X2], axis=1)
    merge6 = ConvLSTM2D(FILTERS, kernel_size=(3,3), padding='same', return_sequences=False, go_backwards=True, kernel_initializer=KERNEL_INIT)(merge6)
    
    conv6 = Conv2D(FILTERS *2, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation(ACTIVATION)(conv6)
    
    conv6 = Conv2D(FILTERS *2, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation(ACTIVATION)(conv6)
    
    up7 = Conv2DTranspose(filters=FILTERS, kernel_size=2, strides=2, padding='same', kernel_initializer=KERNEL_INIT)(conv6)
    up7 = BatchNormalization()(up7)
    up7 = Activation(ACTIVATION)(up7)
    
    X1 = Reshape(target_shape=(1, np.int32(H), np.int32(W), FILTERS))(conv1)
    X2 = Reshape(target_shape=(1, np.int32(H), np.int32(W), FILTERS))(up7)
#     X1 = Reshape(target_shape=(1, np.int32(H), np.int32(W), 32))(conv1)
#     X2 = Reshape(target_shape=(1, np.int32(H), np.int32(W), 32))(up7)
    
    merge7 = concatenate([X1,X2], axis=1)
    merge7 = ConvLSTM2D(FILTERS, kernel_size=(3,3), padding='same', return_sequences=False, go_backwards=True, kernel_initializer=KERNEL_INIT)(merge7)
    
    conv7 = Conv2D(FILTERS, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation(ACTIVATION)(conv7)
    
    conv7 = Conv2D(FILTERS , kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation(ACTIVATION)(conv7)
    
    conv7 = Conv2D(2, kernel_size=(3,3), padding='same', kernel_initializer=KERNEL_INIT)(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation(ACTIVATION)(conv7)
    
    output = Conv2D(1, kernel_size=(1,1), activation='sigmoid')(conv7)
    
    return Model(inputs, output)
    
    

In [14]:
model = build_BCDU_Net(filters=32, activation='relu', kernel_initializer='he_uniform', input_shape=(240,320,1))
model.summary()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 240, 320, 1) 0                                            
__________________________________________________________________________________________________
conv2d_64 (Conv2D)              (None, 240, 320, 32) 320         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 240, 320, 32) 128         conv2d_64[0][0]                  
__________________________________________________________________________________________________
activation_72 (Activation)      (None, 240, 320, 32) 0           batch_normalization_72[0][0]     
____________________________________________________________________________________________